In [ ]:
# Install required packages
!pip install -r requirements.txt > /dev/null 2>&1

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, load_tool, tool
from Gradio_UI import GradioUI
from typing import Any, Optional
#from langchain.tools import tool
import datetime
import requests
import pytz
import yaml
import os
import dotenv
import sys
sys.path.append('./tools')
from final_answer import FinalAnswerTool

In [ ]:
# Set the os environment variable for the Huggingface API token
dotenv.load_dotenv() 
os.environ["HF_API_TOKEN"] = os.getenv("HF_API_TOKEN")

In [ ]:
@tool
def get_flight_information(
    fromId: str, 
    toId: str, 
    departDate: str, 
    returnDate: Optional[str] = None, 
    adults: Optional[int] = 1
) -> str:
    """
    Retrieve flight options for a given route and date, returning a concise summary.

    Args:
        fromId: A string representing the departure airport code (e.g., "SYR.AIRPORT").
        toId: A string representing the arrival airport code (e.g., "EWR.AIRPORT").
              (Ensure the code format matches what the API expects.)
        departDate: A string representing the departure date in YYYY-MM-DD (or MM/DD/YYYY) format.
        returnDate: (Optional) A string representing the return date in the same format.
        adults: (Optional) An integer representing the number of adult passengers. Defaults to 1.

    Returns:
        A concise summary of the top three flight offers (departure time, arrival time, airline, price)
        or an error message.
    """
    import json

    url = "https://booking-com15.p.rapidapi.com/api/v1/flights/searchFlights"
    api_key = os.getenv("RAPIDAPI_KEY")  # Ensure your RapidAPI key is set in your environment variables

    if not api_key:
        return "Error: No RapidAPI key found. Please set the environment variable 'RAPIDAPI_KEY'."

    # Build the querystring using the provided parameters
    querystring = {
        "fromId": fromId,
        "toId": toId,
        "departDate": departDate,
        "pageNo": "1",
        "adults": str(adults),
        "sort": "BEST",
        "cabinClass": "ECONOMY",  # You can parameterize this if needed
        "currency_code": "AED"    # Change to "USD" or your preferred currency if required
    }
    if returnDate:
        querystring["returnDate"] = returnDate

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "booking-com15.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()
        raw_response = response.text

        # Parse and summarize the flight offers
        json_data = json.loads(raw_response)
        offers = json_data.get("data", {}).get("flightOffers", [])
        if not offers:
            return "No flight offers found."

        summary_lines = []
        # Process only the top three offers
        for offer in offers[:3]:
            # Get the first segment details
            segments = offer.get("segments", [])
            if not segments:
                continue
            segment = segments[0]
            departure_time = segment.get("departureTime", "N/A")
            arrival_time = segment.get("arrivalTime", "N/A")
            # Get airline from the first leg (if available)
            legs = segment.get("legs", [])
            if legs:
                first_leg = legs[0]
                carriers_data = first_leg.get("carriersData", [])
                airline = carriers_data[0].get("name", "Unknown Airline") if carriers_data else "Unknown Airline"
            else:
                airline = "Unknown Airline"
            # Get price details from priceBreakdown->total
            price_info = offer.get("priceBreakdown", {}).get("total", {})
            currency = price_info.get("currencyCode", "AED")
            units = price_info.get("units", 0)
            nanos = price_info.get("nanos", 0)
            # Format price as currency, e.g. "AED 949.36" (using first two digits of nanos)
            price = f"{currency} {units}.{str(nanos)[:2]}"
            summary_line = f"Airline: {airline}, Dep: {departure_time}, Arr: {arrival_time}, Price: {price}"
            summary_lines.append(summary_line)
        return "\n".join(summary_lines)
    except Exception as e:
        return f"Error parsing flight data: {str(e)}"

In [ ]:
@tool
def get_tourist_attractions(city: str) -> str:
    """Retrieves a list of top tourist attractions for a given city.
    
    Args:
        city: The name of the city for which tourist attractions should be retrieved.
    
    Returns:
        A formatted string containing a list of top tourist attractions in the specified city.
    """
    search_url = f"https://api.duckduckgo.com/?q=top+tourist+attractions+in+{city}&format=json"
    
    try:
        response = requests.get(search_url)
        response.raise_for_status()  # Catch HTTP errors
        data = response.json()
        
        if "RelatedTopics" in data and data["RelatedTopics"]:
            results = [topic["Text"] for topic in data["RelatedTopics"] if "Text" in topic]
            if results:
                return "\n".join(results[:10])
    
        return f"No tourist attractions found for {city}."
    
    except requests.RequestException as e:
        return f"Error fetching tourist attractions: {str(e)}"
    
@tool
def get_hotels(city: str) -> str:
    """Get top hotels in the given city.

    Args:
        city: The name of the city to search for hotels.

    Returns:
        A formatted string containing a list of top hotels in the specified city.
    """
    search_url = f"https://api.duckduckgo.com/?q=best hotels in {city}&format=json"
    response = requests.get(search_url)
    data = response.json()

    if "RelatedTopics" in data and data["RelatedTopics"]:
        results = [topic["Text"] for topic in data["RelatedTopics"] if "Text" in topic]
        if results:
            return "\n".join(results[:10])
    return f"No hotel information found for {city}."

@tool
def get_local_restaurants(city: str) -> str:
    """Get top restaurants in the given city.

    Args:
        city: The name of the city to search for restaurants.

    Returns:
        A formatted string containing a list of top restaurants in the specified city.
    """
    search_url = f"https://api.duckduckgo.com/?q=best restaurants in {city}&format=json"
    response = requests.get(search_url)
    data = response.json()

    if "RelatedTopics" in data and data["RelatedTopics"]:
        results = [topic["Text"] for topic in data["RelatedTopics"] if "Text" in topic]
        if results:
            return "\n".join(results[:10])
    return f"No restaurant information found for {city}."

@tool
def get_currency_exchange_rate(destination_country: str) -> str:
    """Get the currency exchange rate for the travel destination country.

    Args:
        destination_country: The name of the country to retrieve exchange rate information.

    Returns:
        A formatted string containing exchange rate details for the specified country.
    """
    search_url = f"https://api.duckduckgo.com/?q={destination_country} currency exchange rate&format=json"
    
    try:
        response = requests.get(search_url)
        response.raise_for_status()
        data = response.json()

        if "RelatedTopics" in data and data["RelatedTopics"]:
            results = [topic["Text"] for topic in data["RelatedTopics"] if "Text" in topic]
            if results:
                return "\n".join(results[:3])
        
        return f"No exchange rate information found for {destination_country}."
    
    except requests.RequestException as e:
        return f"Error fetching exchange rate information: {str(e)}"

@tool
def get_wikipedia_information(location: str) -> str:
    """Get a Wikipedia summary about a given location.

    Args:
        location: The name of the location to retrieve Wikipedia information about.

    Returns:
        A formatted string containing a brief Wikipedia summary of the specified location.
    """
    
    try:
        search_url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={location}&format=json"
        response = requests.get(search_url)
        response.raise_for_status()
        
        data = response.json()

        if "query" in data and "search" in data["query"]:
            search_results = data["query"]["search"]
            if search_results:
                title = search_results[0]["title"]
                snippet = search_results[0]["snippet"]
                return f"Title: {title}\nSummary: {snippet}"
        
        return f"No Wikipedia information found for '{location}'."
    
    except requests.RequestException as e:
        return f"Error fetching Wikipedia information: {str(e)}"

# Initialize final_answer tool
final_answer = FinalAnswerTool()


In [ ]:
model = HfApiModel(
max_tokens=2096,
temperature=0.5,
model_id='Qwen/Qwen2.5-Coder-32B-Instruct',
custom_role_conversions=None,
)

In [ ]:
# Import tool from Hub
image_generation_tool = load_tool("agents-course/text-to-image", trust_remote_code=True)

In [ ]:
with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)
    
agent = CodeAgent(
    model=model,
    tools=[
        get_tourist_attractions,
        get_hotels,
        get_local_restaurants,
        get_flight_information,
        get_currency_exchange_rate,
        get_wikipedia_information,
        final_answer,
    ],
    max_steps=8,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    prompt_templates=prompt_templates
)

In [ ]:
GradioUI(agent).launch()